# Qdislib Notebook
## Circuit Cutting Algorithm

Import the PyCOMPSs library

In [ ]:
import pycompss.interactive as ipycompss

Initialize COMPSs runtime. Parameters indicates if the execution will generate task graph, tracefile, monitor interval and debug information.

In [ ]:
import os

if "BINDER_SERVICE_HOST" in os.environ:
    ipycompss.start(
        graph=True,
        project_xml="../xml/project.xml",
        resources_xml="../xml/resources.xml",
    )
else:
    ipycompss.start(graph=True, monitor=1000)  # debug=True, trace=True

Import task and compss_wait_on module before annotating functions or methods

In [ ]:
from pycompss.api.task import task
from pycompss.api.api import compss_wait_on

Import **qibo** module in order to handle quantic circuits.
Import **qiboconnection** module to connect to the Quantum Computer.

In [ ]:
import numpy as np
import qibo
from qibo import models, gates, hamiltonians, callbacks
from qibo.models import Circuit
from qibo.symbols import X, Y, Z, I

from qiboconnection.connection import ConnectionConfiguration
from qiboconnection.api import API

qibo.__version__

Import Qdislib where the algorithm circuit cutting is implemented

In [ ]:
import Qdislib
from Qdislib import wire_cutting as wc
from Qdislib import gate_cutting as gc
from Qdislib import optimal_cut as oc

Credentials needed to connect to the Quantum Computer

In [ ]:
# Insert your credentials here
configuration = ConnectionConfiguration(
    username="bsc-training",
    api_key="b85b702e-6f94-42be-a876-89a670ebd9f6",
)
connection = API(configuration=configuration)
connection.ping()

In [ ]:
# connection.select_device_ids(device_ids=[9])
# connection.list_devices()

Define the circuit you want to cut

In [ ]:
def entire_circuit():
    nqubits = 10
    circuit = models.Circuit(nqubits)

    circuit.add(gates.H(0))
    circuit.add(gates.CZ(0, 1))
    circuit.add(gates.CZ(2, 6))
    circuit.add(gates.RZ(8, np.pi / 3))

    circuit.add(gates.RY(3, np.pi / 5))
    circuit.add(gates.RX(4, np.pi / 5))
    circuit.add(gates.CZ(0, 2))
    circuit.add(gates.CZ(5, 9))

    circuit.add(gates.CZ(3, 5))
    circuit.add(gates.CZ(3, 4))
    circuit.add(gates.CZ(6, 7))
    circuit.add(gates.RY(7, np.pi / 5))
    circuit.add(gates.RZ(1, np.pi / 5))

    circuit.add(gates.CZ(1, 5))
    circuit.add(gates.RX(6, np.pi / 5))
    circuit.add(gates.CZ(7, 8))

    circuit.add(gates.H(9))
    return circuit


circuit = entire_circuit()
print(circuit.draw())

## Algorithms for Wire Cutting

Use the functions implemented to cut and calculate the expected value of the main circuit.

5 functions:

* **circuit_cutting:**  Implements the whole algorithm in a single function. Cuts the circuit in 2 and calculates the expected value of the reconstruction.

In [ ]:
circuit = entire_circuit()
# wc.circuit_cutting("ZZZZZZZZZZ",circuit, 4)

* **split** : Splits a circuit in two subcircuits. Cuts after the gate we pass as the parameter.

In [ ]:
circuit = entire_circuit()
qubit, list_subcircuits = wc.split(circuit, (2, 13), True)

* **simulation** : Performs the execution of a cirucuit to calculate the expected value. It accepts one or two circuits. With 1 circuit it calculates the expected value straight forward, with 2 it performs a reeconstruction in order to provie the expected value.

In [ ]:
circuit = entire_circuit()
# wc.simulation("ZZZZ", circuit)
print("\n")

circuit = entire_circuit()
qubit, list_subcircuits = wc.split(circuit, (2, 13), True)
wc.simulation("ZZZZZZZZZZ", qubit, list_subcircuits[0], list_subcircuits[1], 90000)

* **quantum_computer_execution** : Sends the execution to the quantum computer to calculate the expected value, instead of performing a simulation. (in process)
  - Interactive
  - Enqueue (returns list job ids)

In [ ]:
# circuit = entire_circuit()
# cc.quantum_computer_interactive("ZZZZ", circuit1, circuit2, connection)
# jobs_id = quantum_computer_enqueue("ZZZZ", circuit1, circuit2, connection)

* **analytical_solution** : Computes the analytical expected value for the circuit. 

In [ ]:
circuit = entire_circuit()
wc.analytical_solution("ZZZZZZZZZZ", circuit)

## Algorithm for Gate Cutting

split_gates
simulation
frequencies
expectation_value
reconstruction

gate_cutting


In [ ]:
def entire_circuit():
    nqubits = 10
    circuit = models.Circuit(nqubits)

    circuit.add(gates.H(0))
    circuit.add(gates.CZ(0, 1))
    circuit.add(gates.CZ(2, 6))
    circuit.add(gates.RZ(8, np.pi / 3))

    circuit.add(gates.RY(3, np.pi / 5))
    circuit.add(gates.RX(4, np.pi / 5))
    circuit.add(gates.CZ(0, 2))
    circuit.add(gates.CZ(5, 9))

    circuit.add(gates.CZ(3, 5))
    circuit.add(gates.CZ(3, 4))
    circuit.add(gates.CZ(6, 7))
    circuit.add(gates.RY(7, np.pi / 5))
    circuit.add(gates.RZ(1, np.pi / 5))

    circuit.add(gates.CZ(1, 5))
    circuit.add(gates.RX(6, np.pi / 5))
    circuit.add(gates.CZ(7, 8))

    circuit.add(gates.H(9))
    return circuit

In [ ]:
circuit = entire_circuit()
subcircuits = gc.split_gates([3, 14], circuit, True)

In [ ]:
expectation_value = []
type_gates = type(circuit.queue[3 - 1])
for subcircuit in subcircuits:
    subcircuit.add(gates.M(*range(subcircuit.nqubits)))
    result = gc._gate_simulation(subcircuit, 90000)
    frequencies = gc._gate_frequencies(result)
    expec = gc.gate_expectation_value(frequencies, 90000)
    expectation_value.append(expec)
expectation_value = compss_wait_on(expectation_value)  # IMPORTANT
reconstruct = gc.gate_reconstruction(type_gates, [3, 14], expectation_value)
print(reconstruct)

In [ ]:
circuit = entire_circuit()
print(circuit.draw())

gc.gate_cutting([3, 14], circuit, 90000, 3, True)

## Optimal Cut

parameters: circuit, num_qubits, num_subcircuits, max_cuts, gate_cut, wire_cut

In [ ]:
circuit = entire_circuit()
oc.optimal_cut(circuit)

Stop COMPSs runtime.

In [ ]:
ipycompss.stop(sync=True)